In [1]:
#from cassandra.cluster import Cluster
import pandas as pd 
import collections
import numpy as np

In [2]:
import csv

col_list = ["movieId", "title","genres"]

df = pd.read_csv("/home/spetz/Downloads/archive/movie.csv", usecols=col_list)
#file.readlines()


In [3]:
df=df.drop(columns=["genres","movieId"])
df

,title
0,Toy Story (1995)
1,Jumanji (1995)
2,Grumpier Old Men (1995)
3,Waiting to Exhale (1995)
4,Father of the Bride Part II (1995)
...,...
27273,Kein Bund für's Leben (2007)
27274,"Feuer, Eis & Dosenbier (2002)"
27275,The Pirates (2014)
27276,Rentun Ruusu (2001)


In [4]:
columns_list = ["userId", "movieId","rating","timestamp"]
dx=pd.read_csv("/home/spetz/Downloads/archive/rating.csv", usecols=columns_list)
dx=dx.sort_values("movieId")
dx

,userId,movieId,rating,timestamp
14162882,97809,1,3.0,2008-06-11 04:47:11
15347420,106140,1,5.0,2013-01-29 03:33:49
15347355,106138,1,3.0,2002-07-31 15:48:53
10170195,70354,1,4.5,2011-02-13 18:55:40
10170255,70355,1,3.5,2008-01-26 16:56:54
...,...,...,...,...
11528488,79570,131254,4.0,2015-03-30 19:32:59
11528489,79570,131256,4.0,2015-03-30 19:48:08
4243883,28906,131258,2.5,2015-03-30 19:56:32
9459645,65409,131260,3.0,2015-03-30 19:57:46


In [5]:
dx=dx.drop(columns=["userId"])
dx


,movieId,rating,timestamp
14162882,1,3.0,2008-06-11 04:47:11
15347420,1,5.0,2013-01-29 03:33:49
15347355,1,3.0,2002-07-31 15:48:53
10170195,1,4.5,2011-02-13 18:55:40
10170255,1,3.5,2008-01-26 16:56:54
...,...,...,...
11528488,131254,4.0,2015-03-30 19:32:59
11528489,131256,4.0,2015-03-30 19:48:08
4243883,131258,2.5,2015-03-30 19:56:32
9459645,131260,3.0,2015-03-30 19:57:46


In [16]:
new_list = list()
dicts={}
dxx = dx[(dx['timestamp'] > '2015-01-01') & (dx['timestamp'] <= '2015-01-14')]
for i in range(len(dxx)):
    ar = dxx.loc[(dx['movieId'] == i ) & (dx['rating'] <= 10)]
    rating=ar.apply("rating").sum()
    movies_num=ar['movieId'].value_counts()
    x=sum(rating/movies_num)
    new_list.append(x)
    first_value = dxx['movieId'].iat[i]
    dicts[i] = new_list[i]
    print("To rating tis taineias me movie id: %d einai :%s" %((i),(x)))
dicts


To rating tis taineias me movie id: 0 einai :0
To rating tis taineias me movie id: 1 einai :3.6984126984126986
To rating tis taineias me movie id: 2 einai :2.9166666666666665
To rating tis taineias me movie id: 3 einai :2.8333333333333335
To rating tis taineias me movie id: 4 einai :4.0
To rating tis taineias me movie id: 5 einai :2.8333333333333335
To rating tis taineias me movie id: 6 einai :4.166666666666667
To rating tis taineias me movie id: 7 einai :2.625
To rating tis taineias me movie id: 8 einai :2.5
To rating tis taineias me movie id: 9 einai :0
To rating tis taineias me movie id: 10 einai :3.25
To rating tis taineias me movie id: 11 einai :2.875
To rating tis taineias me movie id: 12 einai :2.4166666666666665
To rating tis taineias me movie id: 13 einai :0
To rating tis taineias me movie id: 14 einai :3.125
To rating tis taineias me movie id: 15 einai :2.75
To rating tis taineias me movie id: 16 einai :4.217391304347826
To rating tis taineias me movie id: 17 einai :4.0526315

{0: 0,
 1: 3.6984126984126986,
 2: 2.9166666666666665,
 3: 2.8333333333333335,
 4: 4.0,
 5: 2.8333333333333335,
 6: 4.166666666666667,
 7: 2.625,
 8: 2.5,
 9: 0,
 10: 3.25,
 11: 2.875,
 12: 2.4166666666666665,
 13: 0,
 14: 3.125,
 15: 2.75,
 16: 4.217391304347826,
 17: 4.052631578947368,
 18: 2.7857142857142856,
 19: 2.78,
 20: 3.75,
 21: 3.75,
 22: 3.1666666666666665,
 23: 3.4166666666666665,
 24: 2.875,
 25: 3.6666666666666665,
 26: 0,
 27: 3.0,
 28: 3.75,
 29: 3.8333333333333335,
 30: 0,
 31: 3.5,
 32: 3.925925925925926,
 33: 0,
 34: 3.0555555555555554,
 35: 3.5,
 36: 4.111111111111111,
 37: 0,
 38: 1.0,
 39: 3.1,
 40: 0,
 41: 3.0,
 42: 4.0,
 43: 0,
 44: 3.0,
 45: 4.5,
 46: 4.0,
 47: 4.138888888888889,
 48: 3.533333333333333,
 49: 0,
 50: 4.182608695652174,
 51: 0,
 52: 2.5,
 53: 0,
 54: 3.0,
 55: 0,
 56: 0,
 57: 2.5,
 58: 0,
 59: 0,
 60: 2.7857142857142856,
 61: 0,
 62: 3.5833333333333335,
 63: 1.8333333333333333,
 64: 0,
 65: 0.5,
 66: 0.5,
 67: 0,
 68: 0,
 69: 3.2,
 70: 3.6470588

In [24]:
def compare_dict_values(dictionary, given_float):
    return {i: dictionary[i] for i in dictionary if dictionary[i] > given_float}


compare_dict_values(dicts,4)

{6: 4.166666666666667,
 16: 4.217391304347826,
 17: 4.052631578947368,
 36: 4.111111111111111,
 45: 4.5,
 47: 4.138888888888889,
 50: 4.182608695652174,
 97: 4.214285714285714,
 111: 4.0638297872340425,
 122: 5.0,
 123: 4.25,
 141: 4.03125,
 147: 4.125,
 203: 5.0,
 222: 5.0,
 233: 4.5,
 242: 4.5,
 293: 4.185897435897436,
 296: 4.2781954887218046,
 306: 4.25,
 307: 4.055555555555555,
 318: 4.248366013071895,
 345: 4.333333333333333,
 363: 5.0,
 417: 4.5,
 421: 4.5,
 431: 4.115384615384615,
 440: 4.1,
 448: 4.166666666666667,
 479: 5.0,
 501: 5.0,
 541: 4.267241379310345,
 549: 4.5,
 608: 4.123376623376624,
 665: 4.5,
 668: 4.5,
 702: 4.5,
 750: 4.192982456140351,
 823: 5.0,
 858: 4.0894308943089435,
 875: 4.5,
 892: 4.5,
 903: 4.148648648648648,
 904: 4.028301886792453,
 905: 4.333333333333333,
 908: 4.041666666666667,
 918: 5.0,
 923: 4.083333333333333,
 924: 4.409090909090909,
 926: 4.5,
 927: 5.0,
 928: 4.3076923076923075,
 931: 4.25,
 933: 4.166666666666667,
 943: 4.75,
 951: 4.1428

In [ ]:
#query1=pd.concat([df, dx], axis=1, ignore_index=False)